In [98]:
import pandas as pd
import numpy as np
from chefboost.training import Training
from sklearn import datasets
df = pd.read_csv("wine_dataset.csv")
df.rename(columns={"type":"Decision"},inplace=True)
config = {"algorithm":"ID3"}
gains = Training.findGains(df,config)
print(gains)
X = df.iloc[:,:-1]
y = df.iloc[:,-1:]

{'gains': {'citric acid': 0.11644331159277607, 'residual sugar': 0.23005216511474283, 'pH': 0.0964537780661524, 'sulphates': 0.17530932492038187, 'alcohol': 0.00873819106050372}, 'entropy': 1.0}


In [84]:
X = pd.DataFrame({"data":[0,0,0,0],"type":["a","B","a","a"]})
y = pd.DataFrame({"data":[1,2,3,4]})
data_is_equal = True
for key in X.keys():
    if len(X[key].unique()) == 1:
        continue
    else:
        data_is_equal = False
print(data_is_equal)

False


In [74]:
entropy_list = []
for key in X.keys():
    split = np.mean(X[key])
    X_split = X[key] < split
    cond_entropy = 0
    for value in X_split.unique():
        prob_X = X_split.value_counts()[value]/len(X_split)
        column = y.keys()[-1]
        for label in y[column].unique():
            prob_y_cond = (((X_split == value)&(y[column] == label)).value_counts()[True]/len(X_split))
            cond_entropy -= prob_y_cond * np.log2(prob_y_cond/prob_X)
    entropy_list.append(cond_entropy)
arr = np.array(entropy_list)
print(arr)
print(1-arr)

[0.99082429 0.76994783 0.90354622 0.82469068 0.99871933]
[0.00917571 0.23005217 0.09645378 0.17530932 0.00128067]


In [6]:
#y = [0,0,0,0,0,0,0,0,0,0]
#y = [1,1,1,1,1,1,1,1,1,1]
y = [0,1,1,1,1,0]
if len(set(y)) == 1:
    print(y[0])

In [85]:
class Node():
        def __init__(self,dec_key,dec_val):
            self.dec_key = dec_key
            self.dec_val = dec_val
        def right(self,next):
            self.right_branch = next
        def left(self,next):
            self.left_branch = next
        pass
class Leaf():
    def __init__(self,val):
        self.val = val
        pass


In [91]:
def calculate_data_entropy(y):
    data_entropy = 0
    column = y.keys()[-1]
    for label in y[column].unique():
        print(y[column].value_counts())
        label_prob = (y[column].value_counts()[label])/len(y)
        data_entropy -= (label_prob) * np.log2(label_prob)
    return data_entropy

In [87]:
def calculate_conditional_entropy(X,y):
    entropy_list = []
    for key in X.keys():
        split = np.mean(X[key])
        X_split = X[key] < split
        cond_entropy = 0
        for value in X_split.unique():
            prob_X = X_split.value_counts()[value]/len(X_split)
            column = y.keys()[-1]
            for label in y[column].unique():
                prob_y_cond = (((X_split == value)&(y[column] == label)).value_counts()[True]/len(X_split))/prob_X
                cond_entropy -= prob_X * prob_y_cond * np.log2(prob_y_cond)
        entropy_list.append(cond_entropy)
    entropy_list = np.array(entropy_list)
    return entropy_list

In [88]:
def information_gain(X,y):
    #calculate entropy of X data set
    data_entropy = calculate_data_entropy(y)
    #calculate conditional entropy given a split data for each predictor
    cond_entropy_array = calculate_conditional_entropy(X,y)
    #difference in data set entropy and conditional entropies gives us information gain of each split
    information_gain = data_entropy - cond_entropy_array
    print(information_gain)
    return np.argmax(information_gain)


In [93]:
def learn(X,y,impurity_measure = 'entropy'):
    print(X.info())
    print(y.info())
    #check if all labels are the same; return leaf of that value if they are
    column = y.keys()[-1]
    if len(y[column].unique()) == 1:
        return Leaf(y[column][0])
    #check if all data values are the same and return most common label if true
    data_is_equal = True
    for key in X.keys():
        if len(X[key].unique()) == 1:
            continue
        else:
            data_is_equal = False
    if data_is_equal:
        return Leaf(y[column].value_counts()[0])
    
    #find split with most information gain and split data to left and right branches
    else:
        split_index = information_gain(X,y)
        split_key = X.keys()[split_index]
        split_value = np.mean(X[split_key])
        branch = Node(split_key,split_value)
        branch.right = learn(X[X[split_key] < split_index].reset_index(),y[X[split_key] < split_index].reset_index())
        branch.left = learn(X[X[split_key] >= split_index].reset_index(),y[X[split_key] >= split_index].reset_index())
        return branch

In [94]:
df = pd.read_csv("wine_dataset.csv")
X = df.iloc[:,:-1]
y = df.iloc[:,-1:]

tree = learn(X,y)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3198 entries, 0 to 3197
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   citric acid     3198 non-null   float64
 1   residual sugar  3198 non-null   float64
 2   pH              3198 non-null   float64
 3   sulphates       3198 non-null   float64
 4   alcohol         3198 non-null   float64
dtypes: float64(5)
memory usage: 125.0 KB
None
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3198 entries, 0 to 3197
Data columns (total 1 columns):
 #   Column  Non-Null Count  Dtype
---  ------  --------------  -----
 0   type    3198 non-null   int64
dtypes: int64(1)
memory usage: 25.1 KB
None
type
1    1599
0    1599
Name: count, dtype: int64
type
1    1599
0    1599
Name: count, dtype: int64
[0.00917571 0.23005217 0.09645378 0.17530932 0.00128067]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 28 entries, 0 to 27
Data columns (total 6 columns):
 #   Column     

KeyError: True